In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
%cd ~/Covid19-LUS/Combined

In [ ]:
data = dict()
data['image_name'] = []
data['labels'] = []
data['image_path'] = []

for label in os.listdir():
    for image in os.listdir(label):
        data['image_name'].append(image)

    for _ in range(len(os.listdir(label))):
        data['labels'].append(label)

    for path in os.listdir(label):
        data['image_path'].append(label+'/'+path)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
def rename_labels1(label):
    if '-' in label:
        label=label.replace('-','')
    if '_' in label:
        label=label.replace('_','')
    if ' ' in label:
        label = label.replace(' ','')

    return label.lower()

def rename_labels2(label):
    if 'ablines' in label:
        label =label.replace('ablines','alines,blines')
    if 'a,blines' in label:
        label =label.replace('a,blines','alines,blines')
    if 'consolidations' in label:
        label = label.replace('consolidations','consolidation')
    
    return label


def combine_f(label):
    x = rename_labels1(label)
    x = rename_labels2(x)
    x =  x.split(',')
    return x

In [ ]:
df['pre_labels']=df.labels.apply(combine_f)

In [ ]:
mlb = MultiLabelBinarizer()
binarized = mlb.fit_transform(df.pre_labels)

In [ ]:
df[mlb.classes_] = binarized

In [ ]:
df[df['blines']==1].drop(['consolidation','effusion'],axis=1)

In [ ]:
df[df['alines']==1].drop(['consolidation','effusion'],axis=1)

In [ ]:
import sklearn
dataframe=pd.concat([df[df['blines']==1].drop(['consolidation','effusion'],axis=1),
          df[df['alines']==1].drop(['consolidation','effusion'],axis=1)
])
dataframe = sklearn.utils.shuffle(dataframe)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from  tensorflow_addons.layers import SpectralNormalization as sn
from tensorflow import keras
from tensorflow.keras import layers
from IPython.display import clear_output
from tensorflow.keras.layers import *